In [1]:
import pandas as pd
import nltk
import re
import contractions
import emoji

from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer

from gensim.corpora import Dictionary
from ast import literal_eval
from gensim.models import LdaMulticore, CoherenceModel
import matplotlib.pyplot as plt
from tqdm import tqdm

## **IMPORT AND PREPROCESS**

In [2]:
df = pd.read_csv("C:/Users/eliza/Documents/Semester VI/ELIZA SEMANGAT/medical_data.csv")
df

C:\Users\eliza\AppData\Local\Temp\ipykernel_34260\173311024.py:1: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("C:/Users/eliza/Documents/Semester VI/ELIZA SEMANGAT/medical_data.csv")


,user_id,username,time,rating,original_text,translated_text,final_with_stopword,final_no_stopword,ori_wc,final_wc,...,category_12,category_13,category_14,category_15,category_16,category_17,category_18,category_19,category_20,category_21
0,1.144028e+20,Sajida Dar,1583977664092,5.0,"Great people,great staff and great facility to...","Great people,great staff and great facility to...",great people great staff and great facility to...,great people great staff great facility go thu...,9,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.027817e+20,Tamara Walker,1604380776309,2.0,Security guard asked me to put a mask on my 13...,Security guard asked me to put a mask on my 13...,security guard ask me to put a mask on my 13 m...,security guard ask put mask 13 month old baby ...,65,68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.147746e+20,Christina Veres,1560489213707,4.0,One of the better Cub foods that I have shoppe...,One of the better Cub foods that I have shoppe...,one of the good cub food that i have shop at e...,one good cub food shop everything need right e...,39,39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.108501e+20,Keegan Leahy,1581384638564,1.0,Front of house has amazing staff.. Owner howev...,Front of house has amazing staff.. Owner howev...,front of house have amazing staff owner howeve...,front house amazing staff owner however lot gr...,51,51,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.106635e+20,Kyle Ebert,1592438884887,2.0,Came in early\nNo one was there except one per...,Came in early\nNo one was there except one per...,come in early no one be there except one perso...,come early no one except one person need simpl...,60,62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316339,1.020788e+20,Kevin Portee,1577891990033,5.0,I feel like a new person on every visit,I feel like a new person on every visit,i feel like a new person on every visit,feel like new person every visit,9,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
316340,1.143790e+20,James Rose,1522868195876,5.0,The aqua therapy I have been receiving there h...,The aqua therapy I have been receiving there h...,the aqua therapy i have be receive there have ...,aqua therapy receive helpful,12,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
316341,1.015381e+20,Maxwell Rosa,1540950220650,5.0,Had baby #1 last year in June. Staff was amaz...,Had baby #1 last year in June. Staff was amaz...,have baby 1 last year in june staff be amaze s...,baby 1 last year june staff amaze shoutout mid...,30,30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
316342,1.028744e+20,Yolonda Taylor,1536770366671,1.0,I gave the staff my authorization number for m...,I gave the staff my authorization number for m...,i give the staff my authorization number for m...,give staff authorization number botox injectio...,116,119,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
#initialize lemmatizer
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))


def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # default to noun
    
#preprocessing function
def preprocess(text):
    if not isinstance(text, str):
        return []
    
    #expand contractions
    text = contractions.fix(text)    
    #remove emoji          
    text = emoji.replace_emoji(text, '')   
    #lowercase    
    text = text.lower()
    #remove url & email
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)
    text = re.sub(r"\S+@\S+", "", text)
    #remove punctuation/number. keep only letters + whitespace
    text = re.sub(r"[^a-z\s]", " ", text)
    
    #replace multiple space/tab/newline with a  single space
    text = re.sub(r"\s+", " ", text).strip()
    
    tokens = word_tokenize(text)
    tagged = pos_tag(tokens)

    processed_tokens = [
        lemmatizer.lemmatize(word, get_wordnet_pos(tag))
        for word, tag in tagged
        if word not in stop_words and len(word) > 1 and word.isalpha()
    ]
    
    return processed_tokens

#apply to dataframe
df["tokens"] = df["translated_text"].apply(preprocess)
df

,user_id,username,time,rating,original_text,translated_text,final_with_stopword,final_no_stopword,ori_wc,final_wc,...,category_13,category_14,category_15,category_16,category_17,category_18,category_19,category_20,category_21,tokens
0,1.144028e+20,Sajida Dar,1583977664092,5.0,"Great people,great staff and great facility to...","Great people,great staff and great facility to...",great people great staff and great facility to...,great people great staff great facility go thu...,9,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[great, people, great, staff, great, facility,..."
1,1.027817e+20,Tamara Walker,1604380776309,2.0,Security guard asked me to put a mask on my 13...,Security guard asked me to put a mask on my 13...,security guard ask me to put a mask on my 13 m...,security guard ask put mask 13 month old baby ...,65,68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[security, guard, ask, put, mask, month, old, ..."
2,1.147746e+20,Christina Veres,1560489213707,4.0,One of the better Cub foods that I have shoppe...,One of the better Cub foods that I have shoppe...,one of the good cub food that i have shop at e...,one good cub food shop everything need right e...,39,39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[one, good, cub, food, shop, everything, need,..."
3,1.108501e+20,Keegan Leahy,1581384638564,1.0,Front of house has amazing staff.. Owner howev...,Front of house has amazing staff.. Owner howev...,front of house have amazing staff owner howeve...,front house amazing staff owner however lot gr...,51,51,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[front, house, amazing, staff, owner, however,..."
4,1.106635e+20,Kyle Ebert,1592438884887,2.0,Came in early\nNo one was there except one per...,Came in early\nNo one was there except one per...,come in early no one be there except one perso...,come early no one except one person need simpl...,60,62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[come, early, one, except, one, person, need, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316339,1.020788e+20,Kevin Portee,1577891990033,5.0,I feel like a new person on every visit,I feel like a new person on every visit,i feel like a new person on every visit,feel like new person every visit,9,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[feel, like, new, person, every, visit]"
316340,1.143790e+20,James Rose,1522868195876,5.0,The aqua therapy I have been receiving there h...,The aqua therapy I have been receiving there h...,the aqua therapy i have be receive there have ...,aqua therapy receive helpful,12,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[aqua, therapy, receive, helpful]"
316341,1.015381e+20,Maxwell Rosa,1540950220650,5.0,Had baby #1 last year in June. Staff was amaz...,Had baby #1 last year in June. Staff was amaz...,have baby 1 last year in june staff be amaze s...,baby 1 last year june staff amaze shoutout mid...,30,30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[baby, last, year, june, staff, amaze, shoutou..."
316342,1.028744e+20,Yolonda Taylor,1536770366671,1.0,I gave the staff my authorization number for m...,I gave the staff my authorization number for m...,i give the staff my authorization number for m...,give staff authorization number botox injectio...,116,119,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[give, staff, authorization, number, botox, in..."


In [4]:
df.to_csv("preprocessing.csv", index = False)

In [5]:
#remove empty or too short reviews
df = df[df["tokens"].str.len() >= 3]

# create a list of the processed tokens for gensim
processed_texts = df["tokens"].tolist()

#create dictionary = mapping of every unique word to a numeric ID
dictionary = Dictionary(df["tokens"])
#filter extremes to improve topic quality
#keep any word that appears in at least 10 documents = extremely rare words will be removed
#keep any words that appears in less than 50% of all documents
#because it is too common, doesnt help in distinguishing topics
dictionary.filter_extremes(no_below = 10, no_above = 0.5)

corpus = [dictionary.doc2bow(text) for text in df["tokens"]]

print(f"Number of unique tokens: {len(dictionary)}")
print(f"Number of documents: {len(corpus)}")

Number of unique tokens: 14890
Number of documents: 310440


# **TRAIN LDA** 
**FIND OPTIMAL K USING COHERENCE SCORE**

In [6]:

def compute_coherence_values(dictionary, corpus, texts, start, limit, step):
    model_list = []
    coherence_scores = {
        'c_v': [],
        'u_mass': [],
        'c_uci': [],
        'c_npmi': []
    }
    
    for num_topics in tqdm(range(start, limit + 1, step), desc = "Training Models"):
        model = LdaMulticore(
            corpus = corpus,
            id2word = dictionary,
            num_topics = num_topics,
            random_state = 42,
            chunksize = 512,
            passes = 10,
            workers = 10
        )
        model_list.append(model)
        
        #save each model independently
        model.save(f"lda_model_k{num_topics}.model")
        
        #compute all coherence
        for metric in coherence_scores:
            cm = CoherenceModel(
                model = model,
                texts = texts,
                dictionary = dictionary,
                corpus = corpus if metric == 'u_mass' else None,
                coherence = metric
            )
            coherence_scores[metric].append(cm.get_coherence())
    return model_list, coherence_scores

In [7]:
start, limit, step = 10, 150, 10
model_list, coherence_scores = compute_coherence_values(dictionary, corpus, processed_texts, start, limit, step)

x = list(range(start, limit + 1, step))
plt.figure(figsize = (12,6))
for metric, scores in coherence_scores.items():
    plt.plot(x, scores, label = metric)
    

plt.xlabel("Number of Topics (K)")
plt.ylabel("Coherence Score")
plt.title("Coherence Metrics for Different Number of Topics")
plt.legend()
plt.grid(True)
plt.show()

Training Models:  93%|█████████▎| 14/15 [7:02:32<44:52, 2692.26s/it]  

: 

: 

In [ ]:
k_values = list(range(start, limit + 1, step))

for i, k in enumerate(k_values):
    print(f"\nK = {k}")
    for metric in coherence_scores:
        print(f"   {metric}: {coherence_scores[metric][i]:.4f}")



K = 10
   c_v: 0.4965
   u_mass: -2.0224
   c_uci: 0.2177
   c_npmi: 0.0343

K = 20
   c_v: 0.4824
   u_mass: -2.4143
   c_uci: 0.1104
   c_npmi: 0.0331

K = 30
   c_v: 0.4685
   u_mass: -2.8911
   c_uci: 0.0027
   c_npmi: 0.0280

K = 40
   c_v: 0.4674
   u_mass: -3.2115
   c_uci: -0.2853
   c_npmi: 0.0207

K = 50
   c_v: 0.4451
   u_mass: -3.4683
   c_uci: -0.4693
   c_npmi: 0.0113

K = 60
   c_v: 0.4224
   u_mass: -3.7351
   c_uci: -0.6487
   c_npmi: 0.0002

K = 70
   c_v: 0.4011
   u_mass: -4.0783
   c_uci: -1.0639
   c_npmi: -0.0158

K = 80
   c_v: 0.4021
   u_mass: -4.1226
   c_uci: -1.1571
   c_npmi: -0.0210

K = 90
   c_v: 0.3852
   u_mass: -4.3060
   c_uci: -1.4037
   c_npmi: -0.0312

K = 100
   c_v: 0.3704
   u_mass: -4.6979
   c_uci: -1.7319
   c_npmi: -0.0459


In [ ]:
def compute_topic_diversity(model, topk):
    """
    Computes topic diversity for an LDA model.
    topk: number of top words to consider per topic
    """
    topic_words = [list(zip(*model.show_topic(i, topn=topk)))[0] for i in range(model.num_topics)]
    all_words = [word for topic in topic_words for word in topic]
    unique_words = set(all_words)
    return len(unique_words) / len(all_words)

diversity_scores = []

for model in model_list:
    score = compute_topic_diversity(model, topk=10)
    diversity_scores.append(score)

k_values = list(range(start, limit + 1, step))

for i, k in enumerate(k_values):
    print(f"K = {k} → Topic Diversity: {diversity_scores[i]:.4f}")



K = 10 → Topic Diversity: 0.7100
K = 20 → Topic Diversity: 0.6950
K = 30 → Topic Diversity: 0.7167
K = 40 → Topic Diversity: 0.7325
K = 50 → Topic Diversity: 0.7820
K = 60 → Topic Diversity: 0.7750
K = 70 → Topic Diversity: 0.8086
K = 80 → Topic Diversity: 0.8087
K = 90 → Topic Diversity: 0.8267
K = 100 → Topic Diversity: 0.8350


**INTERPRET THE BEST MODEL**

In [ ]:
# best_k = 10
# bestmodel = LdaMulticore.load(f"lda_model_k{best_k}.model")

**INTERPRET THE LAST SAVED MODEL**

In [ ]:
# num_top_words = 10  
# for topic_id in range(model.num_topics):
#     print(f"\nTopic {topic_id}:")
#     for word, prob in model.show_topic(topic_id, topn=num_top_words):
#         print(f"   {word} ({prob:.4f})")



Topic 0:
   walk (0.1352)
   horrible (0.1325)
   right (0.0893)
   door (0.0804)
   away (0.0672)
   turn (0.0384)
   hair (0.0244)
   get (0.0235)
   ignore (0.0232)
   moment (0.0174)

Topic 1:
   eye (0.1822)
   contact (0.1170)
   drop (0.0988)
   yesterday (0.0840)
   glass (0.0621)
   window (0.0442)
   frame (0.0231)
   vision (0.0203)
   layout (0.0187)
   lens (0.0150)

Topic 2:
   poor (0.1384)
   multiple (0.1035)
   avoid (0.0947)
   miss (0.0803)
   account (0.0644)
   general (0.0432)
   correctly (0.0312)
   occasion (0.0287)
   error (0.0278)
   condescend (0.0275)

Topic 3:
   dentist (0.0737)
   teeth (0.0522)
   tooth (0.0356)
   go (0.0351)
   get (0.0332)
   cleaning (0.0307)
   dental (0.0268)
   need (0.0236)
   work (0.0224)
   do (0.0195)

Topic 4:
   procedure (0.1862)
   do (0.0585)
   surgeon (0.0406)
   dr (0.0349)
   scar (0.0331)
   cancer (0.0260)
   perfect (0.0231)
   surgery (0.0213)
   size (0.0201)
   aid (0.0193)

Topic 5:
   look (0.2606)
   new

In [ ]:
# def assign_dominant_topic(ldamodel, corpus):
#     dominant_topics = []
#     for doc_bow in corpus:
#         topic_probs = ldamodel.get_document_topics(doc_bow)
#         if topic_probs:
#             dominant_topic = max(topic_probs, key=lambda x: x[1])[0]
#         else:
#             dominant_topic = -1 
#         dominant_topics.append(dominant_topic)
#     return dominant_topics

# df["dominant_topic"] = assign_dominant_topic(model, corpus)
# df



ValueError: Length of values (364266) does not match length of index (316344)

In [ ]:
# Show how many documents had no dominant topic
# print(df["dominant_topic"].isna().sum())


0


In [ ]:
# Show all rows where dominant_topic is NaN
# df[df["dominant_topic"].isna()]


,user_id,username,rating,original_text,gmap_id,business_name,translated_text,wcount,sentiment_label,sentiment_score,...,longitude,num_of_reviews,price,relative_results,resp,state,time,url,tokens,dominant_topic


In [15]:
df["dominant_topic"] = df["dominant_topic"].fillna(-1).astype(int)


C:\Users\eliza\AppData\Local\Temp\ipykernel_31628\1322573224.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["dominant_topic"] = df["dominant_topic"].fillna(-1).astype(int)


In [16]:
df["dominant_topic"] = df["dominant_topic"].astype(int)


C:\Users\eliza\AppData\Local\Temp\ipykernel_31628\4248078682.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["dominant_topic"] = df["dominant_topic"].astype(int)


In [17]:
df

,user_id,username,rating,original_text,gmap_id,business_name,translated_text,wcount,sentiment_label,sentiment_score,...,longitude,num_of_reviews,price,relative_results,resp,state,time,url,tokens,dominant_topic
0,1.056987e+20,Michael Reagan,5.0,I like the professional but also friendly staf...,0x8858272f5a118b01:0xaa680edfef593625,AFC Urgent Care - Bon Secours - Simpsonville,I like the professional but also friendly staf...,46,POSITIVE,0.998923,...,-82.252091,336,NaN,"['0x88582728c3e358ad:0xce7d959d5da763d4', '0x8...",NaN,southcarolina,1599407102726,https://www.google.com/maps/place//data=!4m2!3...,"[like, professional, also, friendly, staff, lo...",38
1,1.054290e+20,Maureen Linker,5.0,I can't give enough praise for Dr K and her wo...,0x54950e4f37f2f0e1:0x2dd2b51972a685db,Prestige Family Dentistry,I can't give enough praise for Dr K and her wo...,94,POSITIVE,0.998932,...,-122.870824,138,NaN,"['0x5495094ec59d9999:0xdcd18a0ff9e13a7c', '0x5...",NaN,oregon,1505351476383,https://www.google.com/maps/place//data=!4m2!3...,"[give, enough, praise, dr, wonderful, staff, g...",13
2,1.019574e+20,Lois A Heit,5.0,"Great experience. My son had an ear infection,...",0x89c81ec194e4521b:0x27ff23adf089401c,Righttime Medical Care,"Great experience. My son had an ear infection,...",22,POSITIVE,0.998928,...,-76.761338,328,NaN,"['0x89c821d187ffe3db:0x2b5bcad6be6af28b', '0x8...",NaN,maryland,1488689562588,https://www.google.com/maps/place//data=!4m2!3...,"[great, experience, son, ear, infection, get, ...",6
3,1.165581e+20,Amy Cronis,5.0,In the new world we are living and learning. ...,0x89e3f4cf374b59ab:0x46ffd1c59cc2ef91,AFC Urgent Care Marlborough,In the new world we are living and learning. ...,25,POSITIVE,0.998819,...,-71.582303,2486,NaN,"['0x89e38c0aa0a29349:0x48add85d6ad900ba', '0x8...",NaN,massachusetts,1596818731062,https://www.google.com/maps/place//data=!4m2!3...,"[new, world, live, learn, afc, urgent, care, m...",63
4,1.042071e+20,Jim Gaudreau,5.0,"From receptionist, to nurses, to doctors..... ...",0x88f5be7e04723443:0x6b5ecbe5dce48f26,Northeast Endoscopy Center,"From receptionist, to nurses, to doctors..... ...",34,POSITIVE,0.998910,...,-71.321756,27,NaN,"['0x89e3911831e4839f:0x49ac18e3568b35b1', '0x8...","{'time': 1628781086310, 'text': ""Jim Gaudreau,...",massachusetts,1628704443573,https://www.google.com/maps/place//data=!4m2!3...,"[receptionist, nurse, doctor, across, board, a...",32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372487,1.080380e+20,MRS. Krystal E. Navarrette,2.0,"slower the molases, if you do not mind waiting...",0x80c8c018f4038b7b:0x6f5080afd059d7ba,Las Vegas Oral Surgery,"slower the molases, if you do not mind waiting...",20,NEGATIVE,0.999476,...,-115.262129,218,NaN,"['0x80c8c018f4038b7b:0x6f5080afd059d7ba', '0x8...",NaN,nevada,1522886664219,https://www.google.com/maps/place//data=!4m2!3...,"[slow, molases, mind, wait, ever, come, hear, ...",91
372490,1.132912e+20,tianna c,1.0,they never answer the phone,0x89c81a067181d489:0x9e49ba8f8f4eec1b,Giant Pharmacy,they never answer the phone,5,NEGATIVE,0.999492,...,-76.724051,18,$$,"['0x89c8170869112547:0x5290820894cb08dc', '0x8...",NaN,maryland,1556844422854,https://www.google.com/maps/place//data=!4m2!3...,"[never, answer, phone]",93
372494,1.117070e+20,Katelyn Dzedulionis,1.0,trash... the worst customer service EVER never...,0x89e37199eafbcba7:0x5c0f6902ee3f62c5,CHA Everett Hospital,trash... the worst customer service EVER never...,9,NEGATIVE,0.999500,...,-71.039229,28,NaN,"['0x89e37199be54432f:0x821312fb868838b9', '0x8...",NaN,massachusetts,1581811904490,https://www.google.com/maps/place//data=!4m2!3...,"[trash, bad, customer, service, ever, never, go]",92
372495,1.117070e+20,Katelyn Dzedulionis,1.0,trash... the worst customer service EVER never...,0x89e37199c6568a0b:0x53d2c35aef9247d7,CHA Everett Hospital,trash... the worst customer service EVER never...,9,NEGATIVE,0.999500,...,-71.039677,98,NaN,"['0x89e37199be54432f:0x821312fb8

In [18]:
from collections import Counter

# Assuming you already have 'dominant_topic' column
topic_counts = Counter(df['dominant_topic'].dropna())
sorted_topics = topic_counts.most_common()  # List of (topic_id, count) sorted by frequency

for topic_id, count in sorted_topics:
    print(f"Topic {topic_id}: {count} documents")


Topic 31: 26029 documents
Topic 13: 25684 documents
Topic 79: 25297 documents
Topic 6: 25250 documents
Topic 38: 21379 documents
Topic 59: 19801 documents
Topic 3: 17599 documents
Topic 48: 16682 documents
Topic 98: 14553 documents
Topic 46: 13468 documents
Topic 91: 13264 documents
Topic 64: 10639 documents
Topic 90: 9711 documents
Topic 47: 9065 documents
Topic 7: 8715 documents
Topic 88: 7568 documents
Topic 92: 6470 documents
Topic 66: 6470 documents
Topic 80: 5890 documents
Topic 51: 5272 documents
Topic 15: 5196 documents
Topic 12: 4901 documents
Topic 30: 4683 documents
Topic 63: 4457 documents
Topic 85: 3839 documents
Topic 68: 3112 documents
Topic 20: 3055 documents
Topic 94: 2635 documents
Topic 73: 2416 documents
Topic 44: 2385 documents
Topic 19: 2337 documents
Topic 61: 2142 documents
Topic 5: 1957 documents
Topic 58: 1812 documents
Topic 99: 1776 documents
Topic 93: 1748 documents
Topic 22: 1740 documents
Topic 11: 1721 documents
Topic 40: 1686 documents
Topic 18: 1600 do

In [19]:
df.to_csv("500topics.csv", index = False)

In [20]:
model.save("lda_500.model")

In [21]:
from gensim.models import LdaMulticore 

model = LdaMulticore.load("lda_500.model")


In [22]:
import pandas as pd

top_n = 10

# Create list of topic-word-score tuples
topic_keywords = []
for topic_id in range(model.num_topics):
    for word, score in model.show_topic(topic_id, topn=top_n):
        topic_keywords.append((topic_id, word, round(score, 4)))

# Convert to DataFrame
topic_df = pd.DataFrame(topic_keywords, columns=["Topic", "Keyword", "Score"])

# Optional: sort by topic then score
topic_df = topic_df.sort_values(by=["Topic", "Score"], ascending=[True, False])
topic_df


,Topic,Keyword,Score
0,0,walk,0.1352
1,0,horrible,0.1325
2,0,right,0.0893
3,0,door,0.0804
4,0,away,0.0672
...,...,...,...
995,99,level,0.0275
996,99,high,0.0260
997,99,equipment,0.0248
998,99,social,0.0213


In [23]:
topic_df.to_csv("lda500_topic_keywords.csv", index=False)
